# Train Quantum-Enhanced GAT Model

This notebook trains a GAT model on quantum-transformed features.

## Overview
1. Load quantum graph with expanded features (182→364 dimensions)
2. Initialize GAT model with quantum features
3. Train with early stopping (or load existing model)
4. Evaluate and save results

## Important Notes
- **Estimated time:** 10-20 minutes for training (or instant if model exists)
- If you encounter errors, try: **Kernel → Restart Kernel** from the menu
- The notebook will automatically load pre-trained model if it exists

In [ ]:
# Clear any cached src imports
import sys
if 'src' in sys.modules:
    del sys.modules['src']
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key]
print("✓ Cleared cached imports")

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import json
import time
from pathlib import Path
import os

# Add project root to sys.path
notebook_dir = os.path.abspath('')
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.models import GAT
from src.config import QUANTUM_MODEL_CONFIG, TRAINING_CONFIG, ARTIFACTS_DIR, ARTIFACT_FILES
from src.utils import set_random_seeds, get_device
from src.train_utils import FocalLoss, compute_class_weights

# Set random seeds
set_random_seeds(TRAINING_CONFIG['random_seed'])
device = get_device()

print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")
print("✓ Imports complete")

Device: cpu


## 1. Import Libraries & Setup

In [ ]:
graph_path = ARTIFACTS_DIR / ARTIFACT_FILES['quantum_graph']
data = torch.load(graph_path, weights_only=False).to(device)
print(f"Quantum graph: {data}")
print(f"Feature dimension: {data.num_node_features}")

# Check if masks exist, otherwise create them
if not hasattr(data, 'train_mask'):
    print("\nCreating train/val/test splits...")
    labeled_indices = torch.where(data.labeled_mask)[0].cpu().numpy()
    labeled_y = data.y[data.labeled_mask].cpu().numpy()
    
    train_val_idx, test_idx = train_test_split(
        labeled_indices, 
        test_size=TRAINING_CONFIG['train_test_split'],
        random_state=TRAINING_CONFIG['random_seed'],
        stratify=labeled_y
    )
    
    train_val_y = data.y[train_val_idx].cpu().numpy()
    train_idx, val_idx = train_test_split(
        train_val_idx,
        test_size=TRAINING_CONFIG['train_val_split'],
        random_state=TRAINING_CONFIG['random_seed'],
        stratify=train_val_y
    )
    
    train_mask = torch.zeros(data.num_nodes, dtype=torch.bool, device=device)
    val_mask = torch.zeros(data.num_nodes, dtype=torch.bool, device=device)
    test_mask = torch.zeros(data.num_nodes, dtype=torch.bool, device=device)
    
    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True
    
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

print(f"\nTrain: {data.train_mask.sum()}, Val: {data.val_mask.sum()}, Test: {data.test_mask.sum()}")

# Check class distribution
train_labels = data.y[data.train_mask]
print(f"\nClass distribution in training set:")
print(f"  Class 0 (Licit): {(train_labels == 0).sum()}")
print(f"  Class 1 (Illicit): {(train_labels == 1).sum()}")
print(f"  Imbalance ratio: {(train_labels == 0).sum().item() / (train_labels == 1).sum().item():.2f}:1")

# Initialize model with quantum-expanded features (use QUANTUM_MODEL_CONFIG for better capacity)
print(f"\n🔧 Initializing GAT model with quantum configuration...")
model = GAT(
    in_channels=data.num_node_features,
    hidden_channels=QUANTUM_MODEL_CONFIG['hidden_channels'],
    out_channels=QUANTUM_MODEL_CONFIG['out_channels'],
    num_heads=QUANTUM_MODEL_CONFIG['num_heads'],
    num_layers=QUANTUM_MODEL_CONFIG['num_layers'],
    dropout=QUANTUM_MODEL_CONFIG['dropout']
).to(device)

print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Optimizer
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr=TRAINING_CONFIG['learning_rate'], 
    weight_decay=TRAINING_CONFIG['weight_decay']
)

# CRITICAL FIX: Address class imbalance with weighted loss
if TRAINING_CONFIG['use_class_weights']:
    class_weights = compute_class_weights(train_labels, num_classes=2).to(device)
    print(f"\n⚖️  Class weights: {class_weights.cpu().numpy()}")
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    print("✓ Using weighted CrossEntropyLoss")
elif TRAINING_CONFIG['use_focal_loss']:
    criterion = FocalLoss(
        alpha=TRAINING_CONFIG['focal_alpha'], 
        gamma=TRAINING_CONFIG['focal_gamma']
    )
    print(f"✓ Using Focal Loss (alpha={TRAINING_CONFIG['focal_alpha']}, gamma={TRAINING_CONFIG['focal_gamma']})")
else:
    criterion = nn.CrossEntropyLoss()
    print("⚠️  Using standard CrossEntropyLoss (no class balancing)")

# Learning rate scheduler
if TRAINING_CONFIG['lr_scheduler']:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max',  # Maximize F1 score
        factor=TRAINING_CONFIG['lr_factor'],
        patience=TRAINING_CONFIG['lr_patience'],
        verbose=True
    )
    print(f"✓ Using ReduceLROnPlateau scheduler (patience={TRAINING_CONFIG['lr_patience']})")
else:
    scheduler = None

print("\n✓ Model, optimizer, and loss function ready")

Quantum graph: Data(x=[203769, 364], edge_index=[2, 672479], y=[203769], timestep=[203769], labeled_mask=[203769], unlabeled_mask=[203769])
Feature dimension: 364

Creating train/val/test splits...

Train: 27938, Val: 9313, Test: 9313
Parameters: 94,470


## 3. Define Training & Evaluation Functions

Training function with gradient descent and evaluation function with standard metrics.

In [ ]:
# Cell 3: Define Training & Evaluation Functions
def train_epoch():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    
    # CRITICAL FIX: Gradient clipping for stable training
    if TRAINING_CONFIG.get('clip_grad_norm'):
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), 
            max_norm=TRAINING_CONFIG['clip_grad_norm']
        )
    
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out[mask].argmax(dim=1)
    prob = F.softmax(out[mask], dim=1)[:, 1]

    y_true = data.y[mask].cpu().numpy()
    y_pred = pred.cpu().numpy()
    y_prob = prob.cpu().numpy()

    # Handle NaN values in predictions
    if np.isnan(y_prob).any():
        print(f"Warning: Found {np.isnan(y_prob).sum()} NaN values in predictions, replacing with 0.5")
        y_prob = np.nan_to_num(y_prob, nan=0.5)
    
    # Ensure no infinite values
    if np.isinf(y_prob).any():
        print(f"Warning: Found {np.isinf(y_prob).sum()} Inf values in predictions, replacing with 0.5")
        y_prob = np.nan_to_num(y_prob, nan=0.5, posinf=1.0, neginf=0.0)

    # Compute metrics with error handling
    try:
        roc_auc = roc_auc_score(y_true, y_prob)
    except (ValueError, RuntimeError) as e:
        print(f"Warning: Could not compute ROC AUC: {e}")
        roc_auc = 0.5

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc
    }

print("✓ Training and evaluation functions defined")

In [ ]:
# Cell 4: Train Quantum GAT Model (or load existing)
import os
from pathlib import Path

model_save_path = ARTIFACTS_DIR / ARTIFACT_FILES['quantum_model']

# Check if model already exists
if Path(model_save_path).exists():
    print(f"⚠️  Model already exists at {model_save_path}")
    print("   Loading existing model instead of retraining...")
    print("   (Delete the file and restart kernel to retrain from scratch)\n")
    
    # Load existing model
    checkpoint = torch.load(model_save_path, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    best_val_f1 = checkpoint['val_f1']
    training_time = 0  # Not trained in this session
    
    print(f"✓ Loaded model from epoch {checkpoint['epoch']} with Val F1: {best_val_f1:.4f}")
    print("✓ Model ready for evaluation")
else:
    print("🚀 Training quantum GAT model with all improvements...")
    print("   - Weighted CrossEntropyLoss for class imbalance")
    print("   - Increased model capacity (128 hidden, 6 heads, 3 layers)")
    print("   - Gradient clipping for stability")
    print("   - Learning rate scheduling")
    print("   - Learnable quantum features\n")
    
    history = {
        'train_loss': [], 
        'val_acc': [], 
        'val_precision': [],
        'val_recall': [],
        'val_f1': [], 
        'val_auc': []
    }
    best_val_f1 = 0
    patience_counter = 0
    EPOCHS = TRAINING_CONFIG['epochs']
    PATIENCE = TRAINING_CONFIG['patience']

    start_time = time.time()

    for epoch in range(1, EPOCHS + 1):
        loss = train_epoch()
        val_metrics = evaluate(data.val_mask)
        
        history['train_loss'].append(loss)
        history['val_acc'].append(val_metrics['accuracy'])
        history['val_precision'].append(val_metrics['precision'])
        history['val_recall'].append(val_metrics['recall'])
        history['val_f1'].append(val_metrics['f1'])
        history['val_auc'].append(val_metrics['roc_auc'])
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch:3d} | Loss: {loss:.4f} | Val F1: {val_metrics['f1']:.4f} | "
                  f"Val Prec: {val_metrics['precision']:.4f} | Val Rec: {val_metrics['recall']:.4f} | "
                  f"Val AUC: {val_metrics['roc_auc']:.4f}")
        
        # Update learning rate scheduler
        if scheduler is not None:
            scheduler.step(val_metrics['f1'])
        
        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            patience_counter = 0
            
            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': best_val_f1,
                'val_metrics': val_metrics,
                'config': QUANTUM_MODEL_CONFIG
            }, model_save_path)
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"\n✓ Early stopping at epoch {epoch}")
                break

    training_time = time.time() - start_time
    
    # Save training history
    results = {
        'training_time': training_time,
        'best_val_f1': best_val_f1,
        'total_epochs': epoch,
        'history': history
    }
    
    results_path = ARTIFACTS_DIR / 'quantum_training_results.json'
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n✅ Training complete in {training_time:.2f}s")
    print(f"✅ Best Val F1: {best_val_f1:.4f}")
    print(f"✅ Results saved to {results_path}")

Training quantum GAT...


ValueError: Input contains NaN.

## 4. Train Quantum GAT Model

Train with early stopping based on validation F1 score. If model already exists, it will be loaded instead of retraining.

In [ ]:
# Load best model for evaluation
checkpoint = torch.load(model_save_path, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ Loaded best model from epoch {checkpoint['epoch']}")

# Evaluate on test set
test_metrics = evaluate(data.test_mask)

print("\n📊 Test Set Performance:")
print(f"  Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"  Precision: {test_metrics['precision']:.4f}")
print(f"  Recall:    {test_metrics['recall']:.4f}")
print(f"  F1 Score:  {test_metrics['f1']:.4f}")
print(f"  ROC AUC:   {test_metrics['roc_auc']:.4f}")

# Save metrics
metrics_path = ARTIFACTS_DIR / ARTIFACT_FILES['quantum_metrics']
with open(metrics_path, 'w') as f:
    json.dump(test_metrics, f, indent=2)

print(f"\n✅ Metrics saved to {metrics_path}")

## 5. Load Best Model & Evaluate

Load the best model checkpoint and evaluate on all splits.

In [ ]:
# Cell 6: Save Metrics to JSON
# Get training time from checkpoint or use 0 if model was pre-loaded
try:
    training_time_value = training_time
except NameError:
    training_time_value = 0

metrics_dict = {
    'model_type': 'GAT_Quantum',
    'training_time': training_time_value,
    'best_epoch': checkpoint['epoch'],
    'performance': {
        'train': train_metrics,
        'val': val_metrics,
        'test': test_metrics
    }
}

metrics_path = ARTIFACTS_DIR / ARTIFACT_FILES['quantum_metrics']
with open(metrics_path, 'w') as f:
    json.dump(metrics_dict, f, indent=2)

print(f"\n✓ Metrics saved to {metrics_path}")
print("\n✅ Quantum GAT training complete!")
print("📊 Proceed to 07_eval_quantum.ipynb to compare with baseline model")

## 6. Save Metrics to JSON

Save all performance metrics for comparison in the next notebook.

## 6. Save Metrics to JSON